In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import pymongo

In [2]:
root_path = 'excel'
df_data = pd.read_excel(root_path+"/DATA_PACHARAPOL.xlsx",skiprows=1,header=None)
df_source = pd.read_excel(root_path+"/rand_viral_v3.xlsx")

In [3]:
columns_head = ['timestamp','cId','gender','age','education','kp6m','sn6m','tr6m','dl6m','oth6m']
columns_txt = ['newsId','hl_positive','hl_negative','hl_anger','hl_anxiety','hl_awe','hl_sadness','hl_surprise','hl_practical','hl_interest'
               ,'t_positive','t_negative','t_anger','t_anxiety','t_awe','t_sadness','t_surprise','t_practical','t_interest']

In [4]:
# คำถามเกี่ยวกับผู้กรอก
df1 = df_data.iloc[:,0:10]
df1.columns = columns_head
# ข่าวที่ 1
df2 = df_data.iloc[:,10:29]
df2.columns = columns_txt
# ข่าวที่ 2
df3 = df_data.iloc[:,29:48]
df3.columns = columns_txt
# ข่าวที่ 3
df4 = df_data.iloc[:,48:67]
df4.columns = columns_txt
# ข่าวที่ 4
df5 = df_data.iloc[:,67:86]
df5.columns = columns_txt

In [5]:
df_ans_concat = pd.concat([df2,df3,df4,df5],ignore_index=True)

In [6]:
# นับจำนวนคนกรอกข่าว
df_cnt = df_ans_concat.iloc[:,0:2].groupby(['newsId'], as_index=False).count()
df_cnt.columns = ['newsId','count']
df_cnt.head()

,newsId,count
0,N013,1
1,N014,1
2,N033,1
3,N051,3
4,N061,1


In [7]:
# หาค่าเฉลี่ยคะแนน
df_avg = df_ans_concat.groupby(['newsId'], as_index=False).mean()
df_avg.head()

,newsId,hl_positive,hl_negative,hl_anger,hl_anxiety,hl_awe,hl_sadness,hl_surprise,hl_practical,hl_interest,t_positive,t_negative,t_anger,t_anxiety,t_awe,t_sadness,t_surprise,t_practical,t_interest
0,N013,4.00,1.00,1.00,1.00,3.00,1.00,2.00,1.00,2.00,4.00,1.00,1.00,1.00,3.00,1.00,3.00,2.00,4.00
1,N014,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,3.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
2,N033,2.00,4.00,2.00,3.00,4.00,4.00,1.00,1.00,1.00,5.00,2.00,1.00,4.00,5.00,3.00,3.00,3.00,3.00
3,N051,4.00,2.00,1.67,2.33,3.67,1.67,3.67,2.00,4.33,4.33,1.67,1.67,1.67,3.67,1.67,4.33,3.00,4.00
4,N061,1.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,3.00,2.00,3.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [8]:
#df_source = df_source[['newsId','hl_polarity','t_polarity','totalView','totalShare','dateDiff','hl_neg_count','hl_pos_count','t_neg_count','t_pos_count','img_count']]
df_source = df_source[['newsId','hl_polarity','t_polarity','totalView','totalShare','img_count']]

In [9]:
df_merge = pd.merge(df_source, df_avg, on='newsId',how='inner')
df_merge.set_index('newsId',inplace=True)

In [10]:
df_merge['hl_polarity'] = df_merge['hl_polarity'].apply(lambda x : 'N' if x=='negative' else 'P')
df_merge['t_polarity'] = df_merge['t_polarity'].apply(lambda x : 'N' if x=='negative' else 'P')

In [11]:
df_merge[0:20]

,hl_polarity,t_polarity,totalView,totalShare,img_count,hl_positive,hl_negative,hl_anger,hl_anxiety,hl_awe,...,hl_interest,t_positive,t_negative,t_anger,t_anxiety,t_awe,t_sadness,t_surprise,t_practical,t_interest
newsId,,,,,,,,,,,,,,,,,,,,,
N214,N,N,124006,1383,6,1.00,3.00,2.00,1.00,1.00,...,1.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,2.00,1.00
N250,N,N,46736,622,5,1.00,1.00,1.00,1.00,1.00,...,3.00,3.00,1.00,1.00,1.00,1.00,2.00,1.00,1.00,3.00
N338,N,N,5848,94,9,2.00,3.00,3.50,3.50,2.00,...,4.00,2.50,3.00,4.00,3.50,2.00,2.50,4.00,4.00,4.50
N092,P,P,37972,387,4,4.00,1.00,1.00,1.00,3.00,...,2.00,5.00,1.00,1.00,1.00,3.00,4.00,3.00,3.00,3.00
N262,N,N,14313,352,6,2.00,2.50,1.50,1.00,1.00,...,1.00,2.00,3.00,2.00,1.00,1.00,1.00,1.00,1.50,1.00
N067,P,P,76096,1464,2,3.00,1.00,1.00,1.00,1.00,...,2.00,1.00,1.00,1.00,1.00,1.00,2.00,3.00,1.00,1.00
N355,N,N,11994,154,5,2.00,5.00,4.00,3.00,4.00,...,1.00,4.00,3.00,3.00,1.00,2.00,1.00,2.00,2.00,1.00
N158,P,P,15239,41,4,4.00,1.00,1.00,1.00,4.00,...,3.00,5.00,1.00,1.00,1.00,3.00,1.00,1.00,1.00,3.00
N332,N,P,19170,132,5,4.00,1.00,1.00,1.00,1.00,...,4.00,4.00,1.00,1.00,1.00,1.00,1.00,3.00,1.00,4.00
